In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import torch
from numpy import genfromtxt
import torch
from deepc_hunt.dynamics import AffineDynamics
from deepc_hunt import DeePC, Trainer

# Temperature Control System

### Load in data

In [2]:
ud = genfromtxt('data/recht_ud.csv', delimiter=',')
yd = genfromtxt('data/recht_yd.csv', delimiter=',')

# Add noise to simulate uncertainty in data
noise_std = 0.1              
yd += np.random.rand(*yd.shape)*noise_std
ud += np.random.rand(*ud.shape)*noise_std

### Initialitse DeePC controller

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

n = 3 # n = number of states
m = 3 # m = number of inputs
p = 3 # p = number of output
q = m+p # q = number of i/o variables
Tini = 4 # Past time horizon                                           
Tf = 10 # Future time horizon         
T = (m+1)*(Tini + Tf + n) - 1    

y_constraints = np.kron(np.ones(Tf), np.array([10,10,10]))
u_constraints = np.kron(np.ones(Tf), np.array([5,5,5]))
q = torch.ones(3)*50
r = torch.ones(3)*2
n_batch = 8

controller = DeePC(
    ud=ud, yd=yd, N=Tf, Tini=Tini, p=3, m=3, n_batch=n_batch, device=device,
    y_constraints=y_constraints, u_constraints=u_constraints,
    stochastic_y=True, stochastic_u=True, linear=True, q=q, r=r
)

controller.initialise(lam_y=1e-3, lam_u=1e-3)
controller.to(device)

67


DeePC(
  (QP_layer): CvxpyLayer()
)

### Get dynamics

In [4]:
A = torch.Tensor([[1.01, 0.01, 0.00], # A - State-space matrix
                  [0.01, 1.01, 0.01], 
                  [0.00, 0.01, 1.01]])

dx = AffineDynamics(A=A, B=torch.eye(3)).to(device)

### Run DeePC-HUNT

In [5]:
epochs = 70
time_steps = 10

# Tune regularization params
deepc_tuner = Trainer(controller=controller, env=dx)
final_params = deepc_tuner.run(epochs=epochs, time_steps=time_steps)

  0%|                                                                        | 0/70 [00:00<?, ?it/s]


ValueError: Inconsistent parameter shapes passed in. Expected parameter 4 to have non-batched shape of (30,) but got torch.Size([8, 60]).